In [3]:
# 05_multihorizon_advanced_ratio_optimized.ipynb
# ---------------------------------------------------------
# PURPOSE:
#   1) Perform a multi-horizon forecast (Close_t+1, t+7, t+30, t+90).
#   2) Use advanced ML (RandomForest, XGBRegressor).
#   3) Add hyperparameter tuning with RandomizedSearchCV.
#   4) Use TimeSeriesSplit for more chronologically-aware CV.
#   5) Evaluate RMSE & MAE on test sets for each horizon.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Regressors and Tools
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

# XGBoost (check availability)
try:
    from xgboost import XGBRegressor
    xgb_available = True
except ImportError:
    xgb_available = False

sns.set_theme(font_scale=1.1)

DATA_FOLDER = "../data"

# Example coin list
all_coins = [
    "ADA", "AVAX", "BCH", "BNB", "BTC",
    "DOGE", "DOT", "ETH", "LEO", "LINK",
    "LTC", "MATIC", "NEAR", "SHIB", "SOL",
    "TON", "TRX", "UNI", "XRP"
]

# Horizons
horizons = ["Close_t+1", "Close_t+7", "Close_t+30", "Close_t+90"]

# Feature columns from your feature engineering
feature_cols = [
    "Close","MA_20","BB_upper","BB_lower","RSI_14",
    "MACD","MACD_Signal","MACD_Hist",
    "Stoch_%K","Stoch_%D"
]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

results = []

# Predefine small parameter distributions for random search
# You can expand these if you want a deeper search.
rf_param_dist = {
    "n_estimators": [100, 200],
    "max_depth": [3, 5, 7, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}

if xgb_available:
    xgb_param_dist = {
        "n_estimators": [100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.1, 0.05],
        "subsample": [1.0, 0.8],
        "colsample_bytree": [1.0, 0.8],
    }

# We'll use a small TimeSeriesSplit for hyperparameter search
# (You can adjust n_splits for more robust search)
tscv = TimeSeriesSplit(n_splits=3)

for coin in all_coins:
    print(f"\n=== Advanced Multi-Horizon (Optimized) for {coin} ===")

    # 1) Load the train/test CSVs from your previous notebooks
    train_csv = f"{coin.lower()}_train_multi.csv"
    test_csv  = f"{coin.lower()}_test_multi.csv"
    train_path = os.path.join(DATA_FOLDER, train_csv)
    test_path  = os.path.join(DATA_FOLDER, test_csv)

    if not (os.path.exists(train_path) and os.path.exists(test_path)):
        print(f"[{coin}] Missing train/test CSV, skipping...")
        continue

    train_df = pd.read_csv(train_path, parse_dates=["Date"])
    test_df  = pd.read_csv(test_path,  parse_dates=["Date"])

    if len(train_df) < 20:
        print(f"[{coin}] Train set <20 rows, skipping coin.")
        continue
    if len(test_df) == 0:
        print(f"[{coin}] Test set empty, skipping coin.")
        continue

    # For each horizon
    for horizon_col in horizons:
        if horizon_col not in train_df.columns:
            continue

        # Local copies to handle dropna per horizon
        train_local = train_df.copy()
        test_local  = test_df.copy()

        # Drop rows missing features or horizon
        train_local.dropna(subset=feature_cols + [horizon_col], inplace=True)
        test_local.dropna(subset=feature_cols + [horizon_col], inplace=True)

        # If no data left
        if len(train_local) < 10 or len(test_local) < 1:
            print(f"{coin} - {horizon_col}: not enough data after dropna, skipping horizon.")
            continue

        X_train = train_local[feature_cols]
        y_train = train_local[horizon_col]
        X_test  = test_local[feature_cols]
        y_test  = test_local[horizon_col]

        print(f"{coin} - {horizon_col}: Train={len(X_train)}, Test={len(X_test)}")

        # -------------------------------------------------
        #  Hyperparameter Tuning for RANDOM FOREST
        # -------------------------------------------------
        rf_base = RandomForestRegressor(random_state=42)
        rf_search = RandomizedSearchCV(
            estimator=rf_base,
            param_distributions=rf_param_dist,
            n_iter=6,  # number of random combos
            scoring="neg_mean_squared_error",  # or "neg_mean_absolute_error"
            cv=tscv,   # time series split
            n_jobs=-1, # use all cores
            random_state=42,
            verbose=0
        )
        # Fit the random search
        rf_search.fit(X_train, y_train)
        best_rf = rf_search.best_estimator_
        # Log the best params
        print(f"   [RF best params] {rf_search.best_params_}")

        # Fit final best RF on the entire train set
        best_rf.fit(X_train, y_train)
        rf_pred = best_rf.predict(X_test)
        rf_rmse_val = rmse(y_test, rf_pred)
        rf_mae_val  = mae(y_test, rf_pred)

        # Feature importances (Top 3)
        rf_importances = best_rf.feature_importances_
        top_feats_rf = sorted(zip(feature_cols, rf_importances), key=lambda x: x[1], reverse=True)[:3]

        # -------------------------------------------------
        #  Hyperparameter Tuning for XGBOOST (if available)
        # -------------------------------------------------
        xgb_rmse_val, xgb_mae_val = np.nan, np.nan
        xgb_best_params = None

        if xgb_available:
            xgb_base = XGBRegressor(random_state=42, use_label_encoder=False, eval_metric="rmse")
            xgb_search = RandomizedSearchCV(
                estimator=xgb_base,
                param_distributions=xgb_param_dist,
                n_iter=6,  # random combos
                scoring="neg_mean_squared_error",
                cv=tscv,
                n_jobs=-1,
                random_state=42,
                verbose=0
            )
            xgb_search.fit(X_train, y_train)
            best_xgb = xgb_search.best_estimator_
            xgb_best_params = xgb_search.best_params_
            print(f"   [XGB best params] {xgb_best_params}")

            # Fit final best XGB
            best_xgb.fit(X_train, y_train)
            xgb_pred = best_xgb.predict(X_test)

            xgb_rmse_val = rmse(y_test, xgb_pred)
            xgb_mae_val  = mae(y_test, xgb_pred)

            # (Optional) XGB feature importances
            xgb_importances = best_xgb.feature_importances_
            top_feats_xgb = sorted(zip(feature_cols, xgb_importances), key=lambda x: x[1], reverse=True)[:3]
        else:
            print("   [XGBoost not installed, skipping XGB model]")

        # Print final metrics
        print(f"   RF => RMSE={rf_rmse_val:.3f}, MAE={rf_mae_val:.3f}")
        if xgb_available:
            print(f"   XGB=> RMSE={xgb_rmse_val:.3f}, MAE={xgb_mae_val:.3f}")

        print(f"     Top RF feats: {top_feats_rf}")
        if xgb_available:
            print(f"     Top XGB feats: {top_feats_xgb}\n")

        results.append({
            "Coin": coin,
            "Horizon": horizon_col,
            "RF_RMSE": rf_rmse_val,
            "RF_MAE": rf_mae_val,
            "XGB_RMSE": xgb_rmse_val,
            "XGB_MAE": xgb_mae_val
        })

# ---------------------------------------------------------
# Convert results to DataFrame and Save
# ---------------------------------------------------------
results_df = pd.DataFrame(results)
out_csv = os.path.join(DATA_FOLDER, "advanced_ratio_multihorizon_results_optimized.csv")
results_df.to_csv(out_csv, index=False)
print(f"\n=== Saved advanced multi-horizon results to: {out_csv} ===")
print(results_df.head(30))




=== Advanced Multi-Horizon (Optimized) for ADA ===
ADA - Close_t+1: Train=2097, Test=525
   [RF best params] {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 7}


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:28:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:28:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


   [XGB best params] {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
   RF => RMSE=0.025, MAE=0.017
   XGB=> RMSE=0.028, MAE=0.019
     Top RF feats: [('Close', np.float64(0.993042887237243)), ('MACD_Signal', np.float64(0.0032221491006848868)), ('MACD', np.float64(0.0014656258874830874))]
     Top XGB feats: [('MA_20', np.float32(0.5321405)), ('Close', np.float32(0.2857514)), ('BB_upper', np.float32(0.1637891))]

ADA - Close_t+7: Train=2097, Test=525
   [RF best params] {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 5}


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:28:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:28:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


   [XGB best params] {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
   RF => RMSE=0.070, MAE=0.049
   XGB=> RMSE=0.076, MAE=0.052
     Top RF feats: [('Close', np.float64(0.9542975897188065)), ('MA_20', np.float64(0.010030532708462757)), ('BB_lower', np.float64(0.009370173455672565))]
     Top XGB feats: [('Close', np.float32(0.50623214)), ('MA_20', np.float32(0.34994417)), ('BB_upper', np.float32(0.09543677))]

ADA - Close_t+30: Train=2097, Test=525
   [RF best params] {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}


...

c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:35:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:35:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


   [XGB best params] {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
   RF => RMSE=0.199, MAE=0.104
   XGB=> RMSE=0.183, MAE=0.084
     Top RF feats: [('Close', np.float64(0.7883632354339881)), ('MACD_Signal', np.float64(0.07706385099560498)), ('MA_20', np.float64(0.03599042078077766))]
     Top XGB feats: [('BB_upper', np.float32(0.46669948)), ('Close', np.float32(0.28166455)), ('MACD_Signal', np.float32(0.09395599))]

XRP - Close_t+30: Train=3312, Test=829
   [RF best params] {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 5}


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:35:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:35:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


   [XGB best params] {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
   RF => RMSE=0.434, MAE=0.206
   XGB=> RMSE=0.442, MAE=0.206
     Top RF feats: [('Close', np.float64(0.7689219090236894)), ('MA_20', np.float64(0.08865914365539748)), ('BB_upper', np.float64(0.07943166986396151))]
     Top XGB feats: [('Close', np.float32(0.36180595)), ('BB_upper', np.float32(0.19902249)), ('Stoch_%K', np.float32(0.14051919))]

XRP - Close_t+90: Train=3312, Test=829
   [RF best params] {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 5}
   [XGB best params] {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
   RF => RMSE=0.759, MAE=0.373
   XGB=> RMSE=0.749, MAE=0.373
     Top RF feats: [('BB_upper', np.float64(0.46655871533033305)), ('MA_20', np.float64(0.3039919793745543)), ('MACD_Hist', np.float64(0.0696250716493035))]
     Top XGB feats: [('BB_upper', np.f

c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:36:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:36:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
